### Bob hat einen Freund Namens Charlie. Bob möchte das die Coins welche er von Alice geschickt bekommt NUR ausgegeben werden können wenn Bob und Charlie die Transaktion mit ihren jeweils eigenen privaten Schlüsseln signieren. Alice muss daher die Coins nicht an den öffentlichen Schlüssel von Bob schicken sondern an ein Skript welches die beiden öffentlichen Schlüssel von Bob und Charlie enthält. Diese Art der Transaktion haben wir bereits als **P2MS**/**Bare Multisig** oder **P2SH** kennengelernt.
Als "Segwit"-Gegenstück zu einer **P2SH-Transaktion** steht die **P2WSH-Transaktion** bei der das Redeemskript und alle Daten zur Erfüllung des Skripts in den Witnessdaten der Transaktion eingetragen werden.
Alice hat jedoch noch nicht auf SegWit aktualisiert. Daher müssen Bob und Charlie ihr Musig P2WSH in ein Legacy P2SH einbetten, welches dann von Alice als Empfänger ihrer Transaktion genutzt werden kann. Wir sprechen daher erneut von einer Nested SegWit oder **P2SH-P2WSH**-Transaktion.

### Aufgrund der Tatsache, dass wir mit einer "leeren" Blockchain beginnen, müssen zunächst ein paar Vorarbeiten geleistet werden.
    1. Verbindung zur Blockchain in regtest Modus aufbauen
    2. Anlegen dreier Nodes und Wallets (Alice, Bob und Charlie)
    3. Mining 101 Blöcke damit Alice Wallet Coins enthält
    4. Funding der Transaktion welche später als Input für die P2SH-P2WSH-Locking Transaktion genutzt wird.
    5. Bob erzeugt seinen öffentlichen Schlüssel an den die Funds gebunden werden sollen
    6. Charlie erzeugt seinen öffentlichen Schlüssel an den die Funds gebunden werden sollen

In [1]:
import sys
sys.path.insert(0, "/media/henning/Volume/Programming/Bitcoin/bitcoin/test/functional")
from test_framework.test_shell import TestShell
from io import BytesIO
from decimal import Decimal

from src.transaction import CTx, CTxIn, CTxOut
from src.script import Script, p2ms_script, p2wsh_script, p2pk_script, p2sh_script
from src.ecdsa import PrivateKey
from src.crypto import sha256, hash160

In [2]:
test = TestShell().setup(num_nodes=3, setup_clean_chain=True)
alice = test.nodes[0]
bob = test.nodes[1]
charlie = test.nodes[2]

alice.createwallet("alice", descriptors=False)
bob.createwallet("bob", descriptors=False)
charlie.createwallet("charlie", descriptors=False)

alice_address_0 = alice.getnewaddress("", "legacy")
test.generatetoaddress(alice, 101, alice_address_0)

alice_address_1 = alice.getnewaddress("", "legacy")
alice_p2sh_p2wsh_funding_tx = alice.sendtoaddress(alice_address_1, 0.00011)
alice_p2sh_p2wsh_funding_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2sh_p2wsh_funding_tx)["hex"])
test.generatetoaddress(alice, 1, alice_address_0)

bob_address_1 = bob.getnewaddress("", "legacy")
bob_private_key_wif_format = bob.dumpprivkey(bob_address_1)
bob_private_key = PrivateKey.convert_wif_format(bob_private_key_wif_format)
bob_private_key_int_format = bob_private_key.get_private_key_int()
bob_public_key = bob_private_key.get_public_key()

charlie_address_1 = charlie.getnewaddress("", "legacy")
charlie_private_key_wif_format = charlie.dumpprivkey(charlie_address_1)
charlie_private_key = PrivateKey.convert_wif_format(charlie_private_key_wif_format)
charlie_private_key_int_format = charlie_private_key.get_private_key_int()
charlie_public_key = charlie_private_key.get_public_key()

2024-03-19T18:08:10.597000Z TestFramework (INFO): PRNG seed is: 109996838949828569
2024-03-19T18:08:10.598000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_uvwov9lt


**1.** Zunächst müssen Bob und Charlie das von ihnen gewünschte Script erzeugen.

In [3]:
bob_and_charlie_script = p2ms_script([bob_public_key.sec_format(), charlie_public_key.sec_format()], 2, 2)

bob_and_charlie_script_sha256 = sha256(bob_and_charlie_script.serialize_script()[1:])
bob_and_charlie_original_p2wsh_script = p2wsh_script(bob_and_charlie_script_sha256)

bob_and_charlie_original_p2wsh_script_hex = bob_and_charlie_original_p2wsh_script.serialize_script().hex()[2:]
bob_and_charlie_original_p2wsh_script_hash160 = hash160(bytes.fromhex(bob_and_charlie_original_p2wsh_script_hex))

**1.1** In der ersten Transaktion werden von Alice die 10000 Satoshis an das P2SH Script von Bob und Charlie versendet. Um das notwendige Transaktionsformular ausfüllen zu können, benötigt Alice folgende Informationen. 

In [4]:
alice_private_key_wif_format = alice.dumpprivkey(alice_address_1)
alice_private_key = PrivateKey.convert_wif_format(alice_private_key_wif_format)
alice_private_key_int_format = alice_private_key.get_private_key_int()
alice_public_key = alice_private_key.get_public_key()

alice_previous_tx_id_to_spent = alice_p2sh_p2wsh_funding_tx_parsed["txid"]

for tx_out in alice_p2sh_p2wsh_funding_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00011000"):
        alice_previous_tx_index_to_spent = tx_out["n"]
        alice_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
alice_amount_to_spent = 10000

**1.2** Nun kann Alice die Lockingtransaktion erzeugen. Um die Satoshis an Bob's Skript zu senden, muss Alice im Output der Transaktion ein scriptPubKey der Form: <Hash160(Bob's and Charlie's Script)> OP_EQUAL.

In [5]:
transaction_input = CTxIn(bytes.fromhex(alice_previous_tx_id_to_spent), alice_previous_tx_index_to_spent, script_sig=Script())

alice_script_pubkey = p2sh_script(bob_and_charlie_original_p2wsh_script_hash160)
transaction_output = CTxOut(alice_amount_to_spent, alice_script_pubkey)

alice_locking_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=False)

print(f"Alice unsignierte P2SH-P2WSH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

Alice unsignierte P2SH-P2WSH Lockingtransaktion: 01000000019150a31f5e850fb56e99e8905eebb188a2447786b5b9a82333358cc58d37486a0000000000ffffffff01102700000000000017a914d153f9ca780f84b7d428f366a9e7034a06935e0387ffffffff


**1.3** Nun muss Alice die Lockingtransaktion noch signieren. D.h. sie muss nachweisen, dass sie aus der vorhergehenden Transaktion berechtigt ist die 10000 Satoshis auszugeben.

In [6]:
alice_script_sig = Script().parse_script(BytesIO(bytes.fromhex(f"{hex(len(alice_previous_script_pub_key_to_spent)//2)[2:]}{alice_previous_script_pub_key_to_spent}")))
alice_locking_transaction.sign_transaction(0, [alice_private_key_int_format], alice_script_sig)

print(f"Alice signierte P2SH-P2WSH Lockingtransaktion: {alice_locking_transaction.serialize_transaction().hex()}")

alice_p2sh_p2wsh_locking_tx = alice.sendrawtransaction(alice_locking_transaction.serialize_transaction().hex())
test.generatetoaddress(alice, 1, alice_address_0)

Alice signierte P2SH-P2WSH Lockingtransaktion: 01000000019150a31f5e850fb56e99e8905eebb188a2447786b5b9a82333358cc58d37486a000000006b483045022100f176970361e823f7dbf3dd10077173e13c3b69b7e69dba3aac627d45ae7ae5aa022053e8de39d41e187d490415768232db98209742f099bf695a7d9f3a666fa75b8e012103cbdcd50888cf05d829452427d72fb3ee6f3e53894b88fd0b83bbd969ca11bafbffffffff01102700000000000017a914d153f9ca780f84b7d428f366a9e7034a06935e0387ffffffff


['799a0b8cbdeed58a3f3f5bc3ab1b7a49814e7ec6240ca640b672807091bba23d']

**2.** Nun möchten Bob und Charlie die an den Hash ihres Scripts gebundenen Satoshis weiter versenden. Dazu müssen die beiden im Witnessfeld der Transaktion, zum Beweis das sie berechtigt sind den Output auszugeben, ihre Signaturen sowie das von ihnen erzeugte Skript vorlegen. Die Reihenfolge in der die Signaturen vorliegen muss die selbe Reihenfolge sein in der die öffentlichen Schlüssel im Skript vorliegen. Damit Bob das notwendige Transaktionsformular ausfüllen kann benötigt er folgende Informationen. Weiterhin müssen die beiden im ScriptSig-Feld des Inputs das Skript (sog. Redeemskript) angeben.

In [7]:
alice_p2ms_locking_tx_parsed = alice.decoderawtransaction(alice.gettransaction(alice_p2sh_p2wsh_locking_tx)["hex"])

bob_previous_tx_id_to_spent = alice_p2ms_locking_tx_parsed["txid"]

for tx_out in alice_p2ms_locking_tx_parsed["vout"]:
    if tx_out["value"] == Decimal("0.00010000"):
        bob_previous_tx_index_to_spent = tx_out["n"]
        bob_previous_script_pub_key_to_spent = tx_out["scriptPubKey"]["hex"]

#Transaktionsoutputdaten
bob_amount_to_spent = 9000
#bob's reciever address is alice public key

**2.1** Nun können Bob und Charlie die Transaktion erstellen um die eben von Alice erhaltenen Satoshis weiter zu versenden.

In [8]:
script_pubkey = p2pk_script(alice_public_key.sec_format())
        
transaction_input = CTxIn(bytes.fromhex(bob_previous_tx_id_to_spent), bob_previous_tx_index_to_spent, script_sig=Script([]))
transaction_output = CTxOut(bob_amount_to_spent, script_pubkey)

bob_spending_transaction = CTx(1, [transaction_input], [transaction_output], 0xffffffff, is_testnet=True, is_segwit=True)

**2.2** Nun können Bob und Charlie diese Transaktion mit ihren privaten Schlüsseln signieren und versenden.

In [9]:
bob_script_sig =  Script.parse_script(BytesIO(bytes.fromhex(f"{hex(len(bob_previous_script_pub_key_to_spent)//2)[2:]}{bob_previous_script_pub_key_to_spent}")))
bob_spending_transaction.sign_transaction(0, [bob_private_key_int_format, charlie_private_key_int_format], bob_script_sig, 2, 2, redeem_script=bob_and_charlie_original_p2wsh_script, input_amount=alice_amount_to_spent, witness_script=bob_and_charlie_script)

print(f"Bob und Charlie signierte P2SH-P2WSH Spendingtransaktion: {bob_spending_transaction.serialize_transaction().hex()}")

bob_p2sh_p2wsh_spending_tx = bob.sendrawtransaction(bob_spending_transaction.serialize_transaction().hex())

print(bob_p2sh_p2wsh_spending_tx)

TestShell().shutdown()

Bob und Charlie signierte P2SH-P2WSH Spendingtransaktion: 010000000001016eec460b251d2d44734ba55f9e9807fa612fa882c12f292835ce05b7f0548c4a0000000023220020435345ef2aa18444f2b6af6619f8491751a373d77f6306989a3e0ec74463cd37ffffffff012823000000000000232103cbdcd50888cf05d829452427d72fb3ee6f3e53894b88fd0b83bbd969ca11bafbac0400483045022100a5d940a9abcc04cfee03a00e7097f990f4c59b6a7e3622480ff74d292f4d5800022023c6155757b416e6a22b9e1f6bdcf4bc43dab200f5dc357f1be02d67c9ef452001483045022100d543397306a860f94b9bd7513e598035749721f26569b8a7f4ab9b5cc27a41a1022058c9dcdc0a7eba56e899d9f05fcc83af111df6a296d037101c0982d632cde1c60147522103130fe16e5ed3cf075b0282d6e26249de25cc426ad37aa7a0d4c1f0b171b4020e21026208f3354dc62777ad9dee9fd885faf34160f2dc25cdc3b5a02e03fa449e712f52aeffffffff
63e2a4fc3e9d13a0bacc186e17c0f413e0ab0cad95a3d11fb99ec6baa9726456
2024-03-19T18:08:13.162000Z TestFramework (INFO): Stopping nodes
2024-03-19T18:08:13.366000Z TestFramework (INFO): Cleaning up /tmp/bitcoin_func_test_uvwov9lt on exit
2024-